# Корреляционный анализ

## Task 1
Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Данные находятся в файле **water.txt**

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('datasets/water.txt', sep='\t')
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [6]:
round(data['mortality'].corr(data['hardness']), 4)

-0.6548

## Task 2
В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.

In [7]:
round(data['mortality'].corr(data['hardness'], method='spearman'), 4)

-0.6317

## Task 3
Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.



In [9]:
north_cities = data[data.location == 'North']
south_cities = data[data.location == 'South']

In [13]:
north_corr = north_cities['mortality'].corr(north_cities['hardness'])
north_corr

-0.3685978383288718

In [14]:
south_corr = south_cities['mortality'].corr(south_cities['hardness'])
south_corr

-0.6021532715484156

In [15]:
round(north_corr, 4)

-0.3686

## Task 4
Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

  $X_1$ \ $X_2$ | **раз в месяц**| **реже** | 
  ------------- | -------------|
  **М**  | 239 | 515 | 
  **Ж**  | 203 | 718 | 
 

In [18]:
a, b, c, d = 239, 515, 203, 718
corr_matt = (a*d - b*c) / ((a+b)*(a+c)*(c+d)*(b+d))**0.5
round(corr_matt, 4)

0.109

## Task 5
В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию **scipy.stats.chi2_contingency**. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).



In [25]:
from scipy.stats import chi2_contingency

In [27]:
observed = [[239, 515], [203, 718]]
chi2_contingency(observed)

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[198.96597015, 555.03402985],
        [243.03402985, 677.96597015]]))

## Task 6
В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.



In [36]:
import scipy

In [73]:
fem = np.append(np.ones(203), np.zeros(718))
mal = np.append(np.ones(239), np.zeros(515))

In [74]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [82]:
left, right = proportions_confint_diff_ind(mal, fem)
round(left, 4)

0.0539

## Task 7
Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [77]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [80]:
z_stat = proportions_diff_z_stat_ind(mal, fem)

In [81]:
2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))

8.153453089576601e-06

## Task 8
Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

 X1\X1|Не доволен|Более или менее|Доволен|
 ----------|---------------|-------|
**Не очень счастлив**|	197|	111|	33|
**Достаточно счастлив**|	382|	685|	331|
**Очень счастлив**|	110|	342|	333|
Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [50]:
observed = [[197, 111, 33], [382, 685, 331], [110, 342, 333]]

In [65]:
chi_stat = chi2_contingency(observed)[0]
round(chi_stat, 4)

293.6831

## Task 9
На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [66]:
chi2_contingency(observed)

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[ 93.08597464, 153.74722662,  94.16679873],
        [381.6251981 , 630.318542  , 386.0562599 ],
        [214.28882726, 353.93423138, 216.77694136]]))

## Task 10
Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

In [69]:
n = sum([sum(l) for l in observed])
v_cramer = np.sqrt(chi_stat/(n*2))
round(v_cramer, 4)

0.2412